This notebook covers the basic of creating UDTFs for manipulating a `GeoPolygon`: A set of one or more rings (closed line strings), with the first representing the shape (external ring) and the rest representing holes in that shape (internal rings)

For example: `POLYGON((0 0,4 0,4 4,0 4,0 0),(1 1, 2 1, 2 2, 1 2,1 1))`

For the API, check the RBC ReadTheDocs page:
* [`Column<GeoPolygon>`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.ColumnGeoPolygon.html#rbc.heavydb.ColumnGeoPolygon)
* [`GeoPolygon`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.GeoPolygon.html#rbc.heavydb.GeoPolygon)

In [359]:
import warnings; warnings.filterwarnings('ignore')

### Connect to the HeavyDB server

In [360]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

GeoPolygon requires HeavyDB 7.0 or newer

In [361]:
heavydb.version[:3]

(7, 0, 0)

### Load test data

In [362]:
from util import load_test_data
from rbc.tests import _PolygonTestTable
table_name = 'polygon_table'
load_test_data(heavydb, _PolygonTestTable, table_name)

List of polygons in `polygon_table`

In [363]:
import pandas as pd
descr, result = heavydb.sql_execute(f'select * from {table_name}')
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,p1,p2,p3,p4
0,"POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(1 2,3 4,2...","POLYGON ((0 0,4.99999995576218 0.0,4.999999955...","POLYGON ((0 0,7 0,7 7,0 7,0 0),(4 4,4 2,2 3,2 ...","POLYGON ((0 0,6 0,6 6,0 6,0 0),(3 3,3 2,2 2,2 ..."
1,"POLYGON ((0 0,5 0,5 5,0 5,0 0))","POLYGON ((0 0,5.99999998044223 0.0,5.999999980...","POLYGON ((0 0,7 0,7 7,0 7,0 0))","POLYGON ((0 0,4 0,4 4,0 4,0 0))"
2,"POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(2 3,1 2,3...","POLYGON ((0 0,4.99999995576218 0.0,4.999999955...","POLYGON ((0 0,6 0,6 6,0 6,0 0),(3 3,3 2,2 2,2 ...","POLYGON ((0 0,7 0,7 7,0 7,0 0),(4 4,4 2,2 3,2 ..."
3,None,None,None,None


### Define a function that operate on GeoPolygon

Function `get_linestring` takes a `Column<GeoPolygon>` as input. It extracts the first `GeoLineString` of each polygon in the column when not null.

In [364]:
@heavydb("int32(TableFunctionManager, Column<Z>, OutputColumn<K>)",
            Z=['GeoPolygon'], K=['GeoLineString'], devices=['cpu'])
def get_linestring(mgr, polygons, linestrings):
    size = len(polygons)
    mgr.set_output_item_values_total_number(0, polygons.get_n_of_values())
    mgr.set_output_row_size(size)
    for i in range(size):
        if polygons.is_null(i):
            linestrings.set_null(i)
        else:
            poly = polygons[i]
            ring = poly[0]
            linestrings[i] = ring
    return size

In [365]:
from pprint import pprint

In [366]:
descr, result = heavydb.sql_execute(f'select p1 from {table_name}')
pprint(list(result))

[('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(1 2,3 4,2 3,1 2))',),
 ('POLYGON ((0 0,5 0,5 5,0 5,0 0))',),
 ('POLYGON ((1 2,3 4,5 6,7 8,9 10,1 2),(2 3,1 2,3 4,2 3),(9 10,7 8,5 6,9 '
  '10))',),
 (None,)]


In [368]:
query = (f'''
    SELECT * FROM
     TABLE(get_linestring(
        cursor(SELECT p1 from {table_name})
    ))
''')

descr, result = heavydb.sql_execute(query)
pprint(list(result))

[('LINESTRING (1 2,3 4,5 6,7 8,9 10)',),
 ('LINESTRING (0 0,5 0,5 5,0 5)',),
 ('LINESTRING (1 2,3 4,5 6,7 8,9 10)',),
 (None,)]
